In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
cd /content/drive/My Drive/Colab/Amozon Sentiment Analysis

/content/drive/My Drive/Colab/Amozon Sentiment Analysis


In [10]:
ls

final_sentiment_analysis.hdf5  sa_encoder.vocab.tokens  Untitled0.ipynb
Running_Flask_in_Colab.ipynb   sentiment_analysis.hdf5


In [11]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.2


In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
import threading

app = Flask(__name__)
padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/Colab/Amozon Sentiment Analysis/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab/Amozon Sentiment Analysis/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6000}).start() 

Model and Vocabalory loaded.......
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:6000/ (Press CTRL+C to quit)


In [13]:
import requests
req = requests.get("http://172.28.0.2:6000/")
print(req.status_code)
print(req.text)

172.28.0.2 - - [07/Jun/2020 03:56:42] "GET / HTTP/1.1" 200 -


200
I am alive!


In [14]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

Still working my way through it but definitely changes your view on investment. Wish it was available on Audible


172.28.0.2 - - [07/Jun/2020 03:57:14] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[0.8370060324668884]],"sentiment ":"positive"}



In [15]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

172.28.0.2 - - [07/Jun/2020 03:57:27] "POST /seclassifier HTTP/1.1" 200 -


I thought this will be good one. But I was wrong
200
{"predictions ":[[-0.8676945567131042]],"sentiment ":"Negative"}



In [16]:
!ps -eaf | grep python

root          19       9  0 03:48 ?        00:00:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         122      19  1 03:49 ?        00:00:09 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-733dfd7d-adab-4f0e-bcde-f1272238038b.json
root         490     122  0 03:57 ?        00:00:00 /bin/bash -c ps -eaf | grep python
root         492     490  0 03:57 ?        00:00:00 grep python


In [17]:
import flask
flask.__version__

'1.1.2'

In [18]:
!pip install flask-ngrok

In [22]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/Colab/Amozon Sentiment Analysis/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab/Amozon Sentiment Analysis/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app.run()

Model and Vocabalory loaded.......
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3bf84cb8f099.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jun/2020 04:25:47] "GET / HTTP/1.1" 200 -


I thought it to be a good product but it is nothing but a piece of crap


127.0.0.1 - - [07/Jun/2020 04:27:01] "POST /seclassifier HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 04:27:30] "POST /seclassifier HTTP/1.1" 200 -


I thought it to be a good product but it is nothing but a piece of crap


127.0.0.1 - - [07/Jun/2020 04:29:13] "POST /seclassifier HTTP/1.1" 200 -


way better than my expectations! atleast for its cheap price
